In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import re
import os
import time
from IterativeLIFModel import *
from SCNNLayer import *

In [2]:
class DVS_CIFAR10(Dataset):
    
    def __init__(self, dataPath, mode):#dataPath = '../../../DATA/DVS-CIFAR10/processed/'
        self.mode = mode
        self.dataPath = dataPath + 'tensor_' + self.mode + '/'
        self.dataFiles = os.listdir(self.dataPath)#self.dataPath = '../../../DATA/DVS-CIFAR10/processed/tensor_train/'
    
    def __getitem__(self, index):
        tensor_file = self.dataFiles[index]
        tensor_data = torch.load(self.dataPath + tensor_file)
        
        label_pattern = r'(\d+)_'
        label = torch.tensor(int(re.findall(label_pattern, tensor_file)[0]), dtype = torch.int64)
        
        return tensor_data, label
    
    def __len__(self):
        return len(self.dataFiles)

In [3]:
dataPath = '../../../DATA/DVS-CIFAR10/processed/'

In [4]:
trainSet = DVS_CIFAR10(dataPath = dataPath, mode = 'train')
trainLoader = DataLoader(dataset = trainSet, batch_size = 5, shuffle=True, num_workers = 0)

In [5]:
testSet = DVS_CIFAR10(dataPath = dataPath, mode = 'test')
testLoader = DataLoader(dataset = testSet, batch_size = 5, shuffle=True, num_workers = 0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
acc_record = []
num_epochs = 100 # max epoch
num_classes = 10
batch_size = 5
names = 'STBPmodelDVS-CIFAR10'

In [8]:
learning_rate = 1e-3

In [9]:
snn = SCNN()
snn.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(snn.parameters(), lr=learning_rate)

In [10]:
# Dacay learning_rate
def lr_scheduler(optimizer, epoch, init_lr=0.1, lr_decay_epoch=50):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    if epoch % lr_decay_epoch == 0 and epoch > 1:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
    return optimizer

In [11]:
for epoch in range(num_epochs):
    train_loss = 0
    start_time = time.time()
    for i, (train_images, train_labels) in enumerate(trainLoader):
        snn.zero_grad()
        optimizer.zero_grad()
        
        train_images = train_images.float().to(device)
        train_predicts = snn(train_images)
        train_labels = torch.zeros(train_labels.shape[0], num_classes).scatter_(1, train_labels.view(-1,1), 1.)
                                            #将index视作列（dim = 1），按照train_labels.view(-1,1)值作为index，把src =“1”插入到zeros中
        loss = criterion(train_predicts.cpu(), train_labels)
        train_loss = loss.item() + train_loss
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 25 == 0:
            print('Epoch [%d/%d], Train Step [%d/%d], Train Loss = %.5f' 
                    %(epoch + 1, num_epochs, i + 1, len(trainSet)//batch_size, train_loss))
            train_loss = 0
            print('Time elasped:', time.time() - start_time)
    correct = 0
    total = 0
    optimizer = lr_scheduler(optimizer, epoch, learning_rate, 40)
    
    with torch.no_grad():
        for j, (test_images, test_targets) in enumerate(testLoader):
            optimizer.zero_grad()
            snn.zero_grad()
            
            test_images = test_images.float().to(device)
            test_predicts = snn(test_images)
            test_labels = torch.zeros(test_targets.shape[0], num_classes).scatter_(1, test_targets.view(-1,1), 1.)
            loss = criterion(test_predicts.cpu(), test_labels)
            
            _, predicted = test_predicts.cpu().max(1)
            total = total + float(test_targets.size(0))
            correct = correct + float(predicted.eq(test_targets).sum().item())
            if (j + 1) % 25 == 0:
                acc = 100. * float(correct)/float(total)
                print('Test Step [%d/%d], Acc: %.5f' %(j + 1, len(testLoader), acc))
    acc = float(100 * correct / total)
    print('Epoch [%d/%d] \t Test Accuracy Over Test Dataset: %.3f' %(epoch + 1, num_epochs, acc))
    acc_record.append(acc)
    if epoch % 5 == 0:
        print('Saving......')
        state = {
            'net': snn.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'acc_record': acc_record,
        }
        if not os.path.isdir('./history'):
            os.mkdir('./history')
        torch.save(state, './history/' + names + '.pk')

Epoch [1/100], Train Step [25/1800], Train Loss = 2.50000
Time elasped: 7.671375513076782
Epoch [1/100], Train Step [50/1800], Train Loss = 2.50000
Time elasped: 13.3862886428833
Epoch [1/100], Train Step [75/1800], Train Loss = 2.50000
Time elasped: 19.14678382873535
Epoch [1/100], Train Step [100/1800], Train Loss = 2.50000
Time elasped: 24.931468963623047
Epoch [1/100], Train Step [125/1800], Train Loss = 2.50000
Time elasped: 30.72650456428528
Epoch [1/100], Train Step [150/1800], Train Loss = 2.50000
Time elasped: 36.50906229019165
Epoch [1/100], Train Step [175/1800], Train Loss = 2.50000
Time elasped: 42.32072830200195
Epoch [1/100], Train Step [200/1800], Train Loss = 2.50000
Time elasped: 48.107956409454346
Epoch [1/100], Train Step [225/1800], Train Loss = 2.61774
Time elasped: 53.84719443321228
Epoch [1/100], Train Step [250/1800], Train Loss = 2.49636
Time elasped: 59.62444829940796
Epoch [1/100], Train Step [275/1800], Train Loss = 2.51414
Time elasped: 65.41258263587952
E

Epoch [2/100], Train Step [375/1800], Train Loss = 2.37509
Time elasped: 88.8756308555603
Epoch [2/100], Train Step [400/1800], Train Loss = 2.27476
Time elasped: 94.79992890357971
Epoch [2/100], Train Step [425/1800], Train Loss = 2.25731
Time elasped: 100.63395476341248
Epoch [2/100], Train Step [450/1800], Train Loss = 2.37063
Time elasped: 106.55624508857727
Epoch [2/100], Train Step [475/1800], Train Loss = 2.27706
Time elasped: 112.42140126228333
Epoch [2/100], Train Step [500/1800], Train Loss = 2.31646
Time elasped: 118.26559543609619
Epoch [2/100], Train Step [525/1800], Train Loss = 2.31253
Time elasped: 124.08640360832214
Epoch [2/100], Train Step [550/1800], Train Loss = 2.28060
Time elasped: 129.96441102027893
Epoch [2/100], Train Step [575/1800], Train Loss = 2.36807
Time elasped: 135.8555941581726
Epoch [2/100], Train Step [600/1800], Train Loss = 2.31138
Time elasped: 141.81879806518555
Epoch [2/100], Train Step [625/1800], Train Loss = 2.25378
Time elasped: 147.8081026

Epoch [3/100], Train Step [725/1800], Train Loss = 2.22860
Time elasped: 172.9453296661377
Epoch [3/100], Train Step [750/1800], Train Loss = 2.22265
Time elasped: 178.77498650550842
Epoch [3/100], Train Step [775/1800], Train Loss = 2.22109
Time elasped: 184.5802092552185
Epoch [3/100], Train Step [800/1800], Train Loss = 2.19158
Time elasped: 190.39356565475464
Epoch [3/100], Train Step [825/1800], Train Loss = 2.20046
Time elasped: 196.2148187160492
Epoch [3/100], Train Step [850/1800], Train Loss = 2.17101
Time elasped: 202.03409910202026
Epoch [3/100], Train Step [875/1800], Train Loss = 2.25815
Time elasped: 207.85904741287231
Epoch [3/100], Train Step [900/1800], Train Loss = 2.19718
Time elasped: 213.65090560913086
Epoch [3/100], Train Step [925/1800], Train Loss = 2.21881
Time elasped: 219.46796536445618
Epoch [3/100], Train Step [950/1800], Train Loss = 2.21196
Time elasped: 225.27768898010254
Epoch [3/100], Train Step [975/1800], Train Loss = 2.23773
Time elasped: 231.092467

Epoch [4/100], Train Step [1075/1800], Train Loss = 2.11519
Time elasped: 250.311505317688
Epoch [4/100], Train Step [1100/1800], Train Loss = 2.12954
Time elasped: 256.127583026886
Epoch [4/100], Train Step [1125/1800], Train Loss = 2.18094
Time elasped: 261.9256544113159
Epoch [4/100], Train Step [1150/1800], Train Loss = 2.22516
Time elasped: 267.77000546455383
Epoch [4/100], Train Step [1175/1800], Train Loss = 2.11503
Time elasped: 273.5892508029938
Epoch [4/100], Train Step [1200/1800], Train Loss = 2.07787
Time elasped: 279.3818736076355
Epoch [4/100], Train Step [1225/1800], Train Loss = 2.12494
Time elasped: 285.23562932014465
Epoch [4/100], Train Step [1250/1800], Train Loss = 2.11124
Time elasped: 291.06907629966736
Epoch [4/100], Train Step [1275/1800], Train Loss = 2.16633
Time elasped: 296.88784193992615
Epoch [4/100], Train Step [1300/1800], Train Loss = 2.10692
Time elasped: 302.687383890152
Epoch [4/100], Train Step [1325/1800], Train Loss = 2.10454
Time elasped: 308.5

Epoch [5/100], Train Step [1425/1800], Train Loss = 2.01240
Time elasped: 337.59266686439514
Epoch [5/100], Train Step [1450/1800], Train Loss = 2.00818
Time elasped: 343.4570560455322
Epoch [5/100], Train Step [1475/1800], Train Loss = 2.01463
Time elasped: 349.45963287353516
Epoch [5/100], Train Step [1500/1800], Train Loss = 2.00156
Time elasped: 355.62211775779724
Epoch [5/100], Train Step [1525/1800], Train Loss = 2.07419
Time elasped: 361.5587077140808
Epoch [5/100], Train Step [1550/1800], Train Loss = 2.00664
Time elasped: 367.5514998435974
Epoch [5/100], Train Step [1575/1800], Train Loss = 1.90093
Time elasped: 373.65408515930176
Epoch [5/100], Train Step [1600/1800], Train Loss = 2.04585
Time elasped: 379.61721634864807
Epoch [5/100], Train Step [1625/1800], Train Loss = 1.99817
Time elasped: 385.5182738304138
Epoch [5/100], Train Step [1650/1800], Train Loss = 1.98645
Time elasped: 391.37301540374756
Epoch [5/100], Train Step [1675/1800], Train Loss = 2.08769
Time elasped: 

...

Epoch [28/100], Train Step [125/1800], Train Loss = 1.48975
Time elasped: 29.26638150215149
Epoch [28/100], Train Step [150/1800], Train Loss = 1.38910
Time elasped: 35.13259243965149
Epoch [28/100], Train Step [175/1800], Train Loss = 1.43445
Time elasped: 41.00562810897827
Epoch [28/100], Train Step [200/1800], Train Loss = 1.43832
Time elasped: 46.888590812683105
Epoch [28/100], Train Step [225/1800], Train Loss = 1.43116
Time elasped: 52.750125885009766
Epoch [28/100], Train Step [250/1800], Train Loss = 1.42990
Time elasped: 58.59671115875244
Epoch [28/100], Train Step [275/1800], Train Loss = 1.47925
Time elasped: 64.44846892356873
Epoch [28/100], Train Step [300/1800], Train Loss = 1.49356
Time elasped: 70.29763865470886
Epoch [28/100], Train Step [325/1800], Train Loss = 1.53154
Time elasped: 76.14094471931458
Epoch [28/100], Train Step [350/1800], Train Loss = 1.64234
Time elasped: 82.00740718841553
Epoch [28/100], Train Step [375/1800], Train Loss = 1.52756
Time elasped: 87.9

Epoch [29/100], Train Step [450/1800], Train Loss = 1.27021
Time elasped: 105.224440574646
Epoch [29/100], Train Step [475/1800], Train Loss = 1.27234
Time elasped: 111.03406929969788
Epoch [29/100], Train Step [500/1800], Train Loss = 1.17346
Time elasped: 116.89639568328857
Epoch [29/100], Train Step [525/1800], Train Loss = 1.31279
Time elasped: 122.7096152305603
Epoch [29/100], Train Step [550/1800], Train Loss = 1.34150
Time elasped: 128.54078435897827
Epoch [29/100], Train Step [575/1800], Train Loss = 1.46690
Time elasped: 134.37783336639404
Epoch [29/100], Train Step [600/1800], Train Loss = 1.23314
Time elasped: 140.19902348518372
Epoch [29/100], Train Step [625/1800], Train Loss = 1.50421
Time elasped: 146.04421305656433
Epoch [29/100], Train Step [650/1800], Train Loss = 1.55529
Time elasped: 151.90740299224854
Epoch [29/100], Train Step [675/1800], Train Loss = 1.69125
Time elasped: 157.77757024765015
Epoch [29/100], Train Step [700/1800], Train Loss = 1.93628
Time elasped:

Epoch [30/100], Train Step [775/1800], Train Loss = 1.33865
Time elasped: 181.8033046722412
Epoch [30/100], Train Step [800/1800], Train Loss = 1.38180
Time elasped: 187.65013074874878
Epoch [30/100], Train Step [825/1800], Train Loss = 1.42185
Time elasped: 193.49023389816284
Epoch [30/100], Train Step [850/1800], Train Loss = 1.43840
Time elasped: 199.35252404212952
Epoch [30/100], Train Step [875/1800], Train Loss = 1.42854
Time elasped: 205.18577933311462
Epoch [30/100], Train Step [900/1800], Train Loss = 1.37731
Time elasped: 211.06028866767883
Epoch [30/100], Train Step [925/1800], Train Loss = 1.36804
Time elasped: 216.93768644332886
Epoch [30/100], Train Step [950/1800], Train Loss = 1.46229
Time elasped: 222.81628108024597
Epoch [30/100], Train Step [975/1800], Train Loss = 1.38034
Time elasped: 228.72481274604797
Epoch [30/100], Train Step [1000/1800], Train Loss = 1.38691
Time elasped: 234.6110384464264
Epoch [30/100], Train Step [1025/1800], Train Loss = 1.37122
Time elasp

Epoch [31/100], Train Step [1100/1800], Train Loss = 1.73620
Time elasped: 257.8212003707886
Epoch [31/100], Train Step [1125/1800], Train Loss = 1.71865
Time elasped: 263.6586756706238
Epoch [31/100], Train Step [1150/1800], Train Loss = 1.73895
Time elasped: 269.5117211341858
Epoch [31/100], Train Step [1175/1800], Train Loss = 1.89096
Time elasped: 275.3615789413452
Epoch [31/100], Train Step [1200/1800], Train Loss = 1.88093
Time elasped: 281.23385429382324
Epoch [31/100], Train Step [1225/1800], Train Loss = 1.82069
Time elasped: 287.1092839241028
Epoch [31/100], Train Step [1250/1800], Train Loss = 1.79710
Time elasped: 292.9886679649353
Epoch [31/100], Train Step [1275/1800], Train Loss = 1.81421
Time elasped: 298.88680028915405
Epoch [31/100], Train Step [1300/1800], Train Loss = 1.80610
Time elasped: 304.7613036632538
Epoch [31/100], Train Step [1325/1800], Train Loss = 1.67973
Time elasped: 310.63299322128296
Epoch [31/100], Train Step [1350/1800], Train Loss = 1.56795
Time e

KeyboardInterrupt: 